# Active Learning
### This is an active learning model which will help me label 500+ emails by using machine learning to learn how i label the emails
### I will manually label only 50 emails then I will run the Model to train it based on my data and it will label the data it self so it can be used for further processing


In [ ]:
!pip install numpy #install numpy from cmd

In [ ]:
!python.exe -m pip install --upgrade pip #upgrade pip

In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


# read the orginal csv file

In [2]:
csv_file='emails.csv'
data=pd.read_csv(csv_file,index_col=False) #reads the original csv file from index 1
data.index=np.arange(1,len(data)+1) #sets the indexing from zero

data.iloc[2] #print a single row

# problematic_emails

from          Radhika Patel <radhikapatel.it@charusat.ac.in>
to         23dcse@charusat.edu.in, 23dce@charusat.edu.in,...
subject    Fwd: Join AI master series by SkillDzire in as...
date                         Fri, 19 Jul 2024 12:41:16 +0530
labels                Inbox,Important,Opened,Category Forums
content    ---------- Forwarded message --------- From: S...
Name: 3, dtype: object

In [3]:
if 'from' not in data.columns or 'content' not in data.columns:
    raise ValueError("CSV file must contain 'from' and 'content' columns.")

In [4]:
rows,column=data.shape
print("the number of rows is {} and the number of columns is {} in dataframe".format(rows,column ))

the number of rows is 577 and the number of columns is 6 in dataframe


In [6]:
partlabeled_csv_file = 'labeled_emails.csv'
recieved=pd.read_csv(partlabeled_csv_file)
# print(f"\nLabeled data saved to {labeled_csv_file}")
recieved

# Save the problematic entries to a separate CSV file for analysis
# problematic_csv_file = 'problematic_emails.csv'
# problematic_emails.to_csv(problematic_csv_file, index=False, encoding='utf-8')
# print(f"\nProblematic emails saved to {problematic_csv_file}")

,from,to,subject,date,labels,content,label
0,NDLI Account Activation <ndl-support@iitkgp.ac...,Vansh <23dcs056@charusat.edu.in>,NDLI user account verification,"Thu, 18 Jul 2024 16:04:06 +0000 (UTC)","Inbox,Important,Opened,Category Updates","NDLI account verification *Dear Vansh ,* Tha...",optional
1,Radhika Patel <radhikapatel.it@charusat.ac.in>,"21dcse@charusat.edu.in, 22dcse@charusat.edu.in...",Regarding the collection of Marksheet (Odd ter...,"Thu, 18 Jul 2024 15:28:13 +0530","Inbox,Important,Opened,Category Forums","Dear Students, Greetings!!! Kindly note that...",priority
2,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dce@charusat.edu.in,...",Fwd: Join AI master series by SkillDzire in as...,"Fri, 19 Jul 2024 12:41:16 +0530","Inbox,Important,Opened,Category Forums",---------- Forwarded message --------- From: S...,optional
3,"""Gaurang Patel (Classroom)"" <no-reply@classroo...",23dcs056@charusat.edu.in,=?UTF-8?Q?New_announcement=3A_=22Dear_students...,"Mon, 15 Jul 2024 03:39:42 -0700","Inbox,Category Updates,Unread",Notification settings CSE202: Microprocessor a...,priority
4,LinkedIn <messages-noreply@linkedin.com>,Vansh Malani <23dcs056@charusat.edu.in>,Kshitish Bhatt commented on your post,"Wed, 17 Jul 2024 13:25:09 +0000 (UTC)","Inbox,Category Social,Unread",View Kshitish’s profile: https://www.linkedin....,optional
...,...,...,...,...,...,...,...
480,Bhargav Shobhana <bhargavshobhana.cv@charusat....,NaN,Important- Online Quiz of Civil Part ME145,"Thu, 11 Apr 2024 10:44:37 +0530","Inbox,Important,Opened,Category Personal","Dear Students, An online quiz is planned for ...",NaN
481,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dce@charusat.edu.in,...",Fwd: Engineering Physics - 2 (PY143) || Lab Ma...,"Fri, 26 Apr 2024 09:56:41 +0530","Inbox,Important,Opened,Category Forums","Regards, Radhika H. Patel, Assistant Professor...",NaN
482,"""Coursera"" <Coursera@m.learn.coursera.org>",23dcs056@charusat.edu.in,NEW! AI content added to Professional Certific...,"Tue, 16 Jul 2024 15:43:58 +0000","Inbox,Category Promotions,Unread","Plus, get a special offer from Google /...",NaN
483,Devpost <support@devpost.com>,23dcs056@charusat.edu.in,Level Up with Devpost's Discord Leaderboard Re...,"Thu, 11 Jul 2024 19:17:12 +0000","Trash,Category Updates,Unread",****************************************** Tak...,NaN


In [7]:
recieved
recieved.index=np.arange(1,len(recieved)+1)
reclen=len(recieved)

for index,row in recieved.iloc[:reclen].iterrows():
    # content = row['content'][:500].encode('utf-8', errors='ignore').decode('utf-8', errors='ignore')
    print(f"\nEmail {index} of {reclen}")
    print("From:", row['from'])


Email 1 of 485
From: NDLI Account Activation <ndl-support@iitkgp.ac.in>

Email 2 of 485
From: Radhika Patel <radhikapatel.it@charusat.ac.in>

Email 3 of 485
From: Radhika Patel <radhikapatel.it@charusat.ac.in>

Email 4 of 485
From: "Gaurang Patel (Classroom)" <no-reply@classroom.google.com>

Email 5 of 485
From: LinkedIn <messages-noreply@linkedin.com>

Email 6 of 485
From: Krishna Patel <krishnapatel.ce@charusat.ac.in>

Email 7 of 485
From: Radhika Patel <radhikapatel.it@charusat.ac.in>

Email 8 of 485
From: Tedx CHARUSAT <tedx@charusat.ac.in>

Email 9 of 485
From: "Gaurang Patel (Classroom)" <no-reply@classroom.google.com>

Email 10 of 485
From: National Digital Library of India <ndl-support@iitkgp.ac.in>

Email 11 of 485
From: Radhika Patel <radhikapatel.it@charusat.ac.in>

Email 12 of 485
From: "Nilesh Dubey (Classroom)" <no-reply@classroom.google.com>

Email 13 of 485
From: "Nilesh Dubey (Classroom)" <no-reply@classroom.google.com>

Email 14 of 485
From: Google <no-reply@accounts

In [9]:


# Prepare data for model training
partlabeled_data = recieved
partlabeled_data['content']=partlabeled_data['content'].fillna("")
df=partlabeled_data
train_data = df[df['label'].notna()]
test_data = df[df['label'].isna()]


In [10]:
# train_data #prints 50 emeails with labels
test_data

,from,to,subject,date,labels,content,label
51,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dit@charusat.edu.in,...",Re: Notice of Fees Payment : Odd Term - AY : 2...,"Thu, 18 Jul 2024 16:56:03 +0530","Inbox,Important,Opened,Category Forums","Dear Students, Greetings!!! Gentle reminder ...",NaN
52,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dit@charusat.edu.in,...",Notice of Fees Payment : Odd Term - AY : 2024-25.,"Fri, 14 Jun 2024 16:07:31 +0530","Inbox,Important,Opened,Category Forums","Dear Students, Greetings!!! PFA... The noti...",NaN
53,Naina Parmar <nainaparmar.dcs@charusat.ac.in>,23dcse@charusat.edu.in,"Stakeholder Feedback on Curriculum F.Y. CSE,20...","Mon, 1 Jan 2024 15:55:11 +0530","Inbox,Important,Opened,Category Forums","Dear students, Greetings!!! Hope you are do...",NaN
54,"""Parth Goel (Classroom)"" <no-reply@classroom.g...",23dcs056@charusat.edu.in,=?UTF-8?Q?New_announcement=3A_=22Dear_Students...,"Fri, 12 Jul 2024 00:07:59 -0700","Inbox,Category Updates,Unread",Notification settings 3CSE - CSE204 Project 1 ...,NaN
55,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dit@charusat.edu.in, 23dcse@charusat.edu.in,...",Fwd: First Internal Examination Syllabus of MA...,"Thu, 22 Feb 2024 09:24:32 +0530","Inbox,Important,Opened,Category Forums","*Thanks and Regards,* *RADHIKA H. PATEL* *Assi...",NaN
...,...,...,...,...,...,...,...
481,Bhargav Shobhana <bhargavshobhana.cv@charusat....,NaN,Important- Online Quiz of Civil Part ME145,"Thu, 11 Apr 2024 10:44:37 +0530","Inbox,Important,Opened,Category Personal","Dear Students, An online quiz is planned for ...",NaN
482,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dce@charusat.edu.in,...",Fwd: Engineering Physics - 2 (PY143) || Lab Ma...,"Fri, 26 Apr 2024 09:56:41 +0530","Inbox,Important,Opened,Category Forums","Regards, Radhika H. Patel, Assistant Professor...",NaN
483,"""Coursera"" <Coursera@m.learn.coursera.org>",23dcs056@charusat.edu.in,NEW! AI content added to Professional Certific...,"Tue, 16 Jul 2024 15:43:58 +0000","Inbox,Category Promotions,Unread","Plus, get a special offer from Google /...",NaN
484,Devpost <support@devpost.com>,23dcs056@charusat.edu.in,Level Up with Devpost's Discord Leaderboard Re...,"Thu, 11 Jul 2024 19:17:12 +0000","Trash,Category Updates,Unread",****************************************** Tak...,NaN


In [11]:

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['content'])
X_test = vectorizer.transform(test_data['content'])

# Train a simple classifier
model = MultinomialNB()
model.fit(X_train, train_data['label'])

# Predict labels for unlabeled data
predicted_labels = model.predict(X_test)
test_data.loc[:, 'label'] = predicted_labels


In [12]:
results = pd.concat([train_data, test_data]).sort_index()
results

,from,to,subject,date,labels,content,label
1,NDLI Account Activation <ndl-support@iitkgp.ac...,Vansh <23dcs056@charusat.edu.in>,NDLI user account verification,"Thu, 18 Jul 2024 16:04:06 +0000 (UTC)","Inbox,Important,Opened,Category Updates","NDLI account verification *Dear Vansh ,* Tha...",optional
2,Radhika Patel <radhikapatel.it@charusat.ac.in>,"21dcse@charusat.edu.in, 22dcse@charusat.edu.in...",Regarding the collection of Marksheet (Odd ter...,"Thu, 18 Jul 2024 15:28:13 +0530","Inbox,Important,Opened,Category Forums","Dear Students, Greetings!!! Kindly note that...",priority
3,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dce@charusat.edu.in,...",Fwd: Join AI master series by SkillDzire in as...,"Fri, 19 Jul 2024 12:41:16 +0530","Inbox,Important,Opened,Category Forums",---------- Forwarded message --------- From: S...,optional
4,"""Gaurang Patel (Classroom)"" <no-reply@classroo...",23dcs056@charusat.edu.in,=?UTF-8?Q?New_announcement=3A_=22Dear_students...,"Mon, 15 Jul 2024 03:39:42 -0700","Inbox,Category Updates,Unread",Notification settings CSE202: Microprocessor a...,priority
5,LinkedIn <messages-noreply@linkedin.com>,Vansh Malani <23dcs056@charusat.edu.in>,Kshitish Bhatt commented on your post,"Wed, 17 Jul 2024 13:25:09 +0000 (UTC)","Inbox,Category Social,Unread",View Kshitish’s profile: https://www.linkedin....,optional
...,...,...,...,...,...,...,...
481,Bhargav Shobhana <bhargavshobhana.cv@charusat....,NaN,Important- Online Quiz of Civil Part ME145,"Thu, 11 Apr 2024 10:44:37 +0530","Inbox,Important,Opened,Category Personal","Dear Students, An online quiz is planned for ...",priority
482,Radhika Patel <radhikapatel.it@charusat.ac.in>,"23dcse@charusat.edu.in, 23dce@charusat.edu.in,...",Fwd: Engineering Physics - 2 (PY143) || Lab Ma...,"Fri, 26 Apr 2024 09:56:41 +0530","Inbox,Important,Opened,Category Forums","Regards, Radhika H. Patel, Assistant Professor...",priority
483,"""Coursera"" <Coursera@m.learn.coursera.org>",23dcs056@charusat.edu.in,NEW! AI content added to Professional Certific...,"Tue, 16 Jul 2024 15:43:58 +0000","Inbox,Category Promotions,Unread","Plus, get a special offer from Google /...",priority
484,Devpost <support@devpost.com>,23dcs056@charusat.edu.in,Level Up with Devpost's Discord Leaderboard Re...,"Thu, 11 Jul 2024 19:17:12 +0000","Trash,Category Updates,Unread",****************************************** Tak...,optional


In [13]:
import time

# Display predictions and allow user to correct them
for index, row in results.iterrows():
    predicted_label = row['label']  # Use the existing predicted label from the DataFrame
    print(f"Email {index}")
    print("From:", row['from'])
    print("Content:", row['content'][:500])  # Limit display to 500 characters for readability
    print(f"Predicted Label: {predicted_label}")
    print("-" * 80)

    # Wait for 4 seconds before moving to the next email
    time.sleep(4)

    # Automatically use the existing label
    label = predicted_label

    # Assign the label to the DataFrame
    results.at[index, 'label'] = label

# Save the labeled data to a new CSV file
labeled_csv_file = 'labeled_emails_active_learning.csv'
results.to_csv(labeled_csv_file, index=False, encoding='utf-8')

print(f'Labeled data saved to {labeled_csv_file}')


Email 1
From: NDLI Account Activation <ndl-support@iitkgp.ac.in>
Content: NDLI account verification  *Dear Vansh ,*  Thank you for registering at National Digital Library of India (NDLI). Click here ( https://u4465288.ct.sendgrid.net/ls/click?upn=u001.60YJKKO-2BdVEKZL1Y4rD3gMUfJ6blBHcGrblo52abLsBZ-2Bt-2Bo3UYBM1W-2BUj2sx5-2BCkCqdD3BNq8mMNz3qXadEggF3Q9QEE-2BtODWml00Dl2LPbR2tDhbHHupvGVcdcuK8e6WC-_poMrW87m23-2FzGYUTgt5VO7S5u6NpYTbgNOQhtOfCLgaIwpjdhrHrN973bD9qNgj4g8nwX5Cufa2I1iEKeQPkgDx6RcvYzq3wVJ1HuGaAomSGo0fHt9oreq9TeMiKVzfzCB-2B-2B7ZY7Epq4SQ3F-2FaK0IQTyp-2FlbOf3hQzfGXwh
Predicted Label: optional
--------------------------------------------------------------------------------
Email 2
From: Radhika Patel <radhikapatel.it@charusat.ac.in>
Content: Dear Students,  Greetings!!!  Kindly note that the marksheets for previous term examinations' (Regular/Backlog) are available at PA to Principal office, Ground floor, DEPSTAR building.  *Please collect the same between 11:00 am to 3:00 pm on Wednes